In [1]:
import json
import pandas as pd
OUT_DIR ="OUT/"
ASSET_DIR="assets/"
with open(ASSET_DIR + "dump-formatted.json", "r", encoding="utf-8") as file:
  RAW_DOCUMENTS = json.load(file)
DOCUMENT_DF = pd.DataFrame.from_records(RAW_DOCUMENTS)
DATA_LEN = len(DOCUMENT_DF)
  

### Text Cleaning Function

In [2]:
import jaconv
import re
def cleantext(text):
  text = jaconv.z2h(text, kana=False, digit=True, ascii=True)
  text = text.replace('\\n', " ").replace("\\r", " ")
  text = re.sub(r'\s+', ' ', text)  
  text = text.strip()
  text = text.lower()
  return text

### Apply cleaning function

In [3]:
DOCUMENT_DF["cleaned_content"] = DOCUMENT_DF["content"].apply(cleantext)

### Initialize all models and tokenizers from IndoBERT and IndoBERTweet

In [4]:
from transformers import AutoModel, AutoTokenizer
indobert_model = AutoModel.from_pretrained("indobenchmark/indobert-base-p2", cache_dir="cache/")
indobert_tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2", cache_dir="cache/")

tweet_model = AutoModel.from_pretrained("indolem/indobertweet-base-uncased", cache_dir="cache/")
tweet_tokenizer = AutoTokenizer.from_pretrained("indolem/indobertweet-base-uncased", cache_dir="cache/")

#Turn on evaluation mode as default
indobert_model.eval()
tweet_model.eval()


/Users/christianharjuno/anaconda3/envs/sentiment/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31923, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

### Create functions to get encodings for both indobert and indobertweet

In [5]:
import torch
def getEncodings(textArray):
  indobert_inputs = indobert_tokenizer(
    textArray,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
  )
  tweet_inputs = tweet_tokenizer(
    textArray,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
  )
  with torch.no_grad():
    indobert_outputs = indobert_model(**indobert_inputs)
    tweet_outputs = tweet_model(**tweet_inputs)
  indobert_embeddings = indobert_outputs.last_hidden_state[:, 0, :]
  tweet_embeddings = tweet_outputs.last_hidden_state[:, 0, :]
  return (indobert_embeddings.cpu().numpy(), tweet_embeddings.cpu().numpy())
  

In [6]:
import os

def removeFile(file_path):
  if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} deleted.")
  else:
    print(f"{file_path} does not exist.")

In [7]:
from tqdm import tqdm
import json
INDOBERT_OUT_FILE = OUT_DIR + "indobert_embeds.jsonl"
INDOBERTWEET_OUT_FILE = OUT_DIR + "indobertweet_embeds.jsonl"
def get_batch_embeddings(documents, batch_size=32):
  #Document is in the shape of a Pandas Dataframe. Convert to a list first before processing
  removeFile(INDOBERT_OUT_FILE)
  removeFile(INDOBERTWEET_OUT_FILE)
  documents_list = documents.to_dict(orient="records")
  for i in tqdm(range(0, len(documents_list), batch_size), desc="Generating embeddings"):
    batched = documents_list[i:i+batch_size]
    texts = [doc["cleaned_content"] for doc in batched]
    indobert_embedding, tweet_embedding = getEncodings(texts)
    
    with open(INDOBERT_OUT_FILE, "a", encoding="utf-8") as file:
      for doc, embed in zip(batched, indobert_embedding):
        doc_copy = doc.copy()
        doc_copy["embedding"] = embed.tolist()
        file.write(json.dumps(doc_copy, ensure_ascii=False)+ "\n")
        
    with open(INDOBERTWEET_OUT_FILE, "a", encoding="utf-8") as file:
      for doc, embed in zip(batched, tweet_embedding):
        doc_copy = doc.copy()
        doc_copy["embedding"] = embed.tolist()
        file.write(json.dumps(doc_copy, ensure_ascii=False)+ "\n")

In [ ]:
get_batch_embeddings(DOCUMENT_DF)

OUT/indobert_embeds.jsonl deleted.
OUT/indobertweet_embeds.jsonl deleted.


Generating embeddings: 100%|██████████| 4/4 [00:04<00:00,  1.04s/it]


### Create function to reduce embed size

In [18]:
import umap
import numpy as np
def reduce_embed_size(embeds):
  umap_model = umap.UMAP(n_components=45)
  reduced_embedding = umap_model.fit_transform(np.array(embeds))
  return reduced_embedding

### Process Indobert Embeddings

In [52]:
import json
import pandas as pd
INDOBERT_REDUCED_OUT_FILE = OUT_DIR + "indobert_reduced_embeds.jsonl"

with open(INDOBERT_OUT_FILE, "r", encoding="utf-8") as file:
  embedded_documents = []
  for line in file:
    doc = json.loads(line)
    embedded_documents.append(doc)

embeddings = [doc["embedding"] for doc in embedded_documents]
reduced_embeddings = reduce_embed_size(embeddings)

for doc, reduced in zip(embedded_documents, reduced_embeddings):
  doc["embedding"] = reduced.tolist()
removeFile(INDOBERT_REDUCED_OUT_FILE)
with open(INDOBERT_REDUCED_OUT_FILE, "a", encoding="utf-8") as file:
  for doc in embedded_documents:
    file.write(json.dumps(doc, ensure_ascii=False) + "\n")

/Users/christianharjuno/anaconda3/envs/sentiment/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


OUT/indobert_reduced_embeds.jsonl deleted.


### Process IndoBERTweet embeddings

In [53]:
import json
import pandas as pd
TWEET_REDUCED_OUT_FILE = OUT_DIR + "indobertweet_reduced_embeds.jsonl"

with open(INDOBERTWEET_OUT_FILE, "r", encoding="utf-8") as file:
  embedded_documents = []
  for line in file:
    doc = json.loads(line)
    embedded_documents.append(doc)

embeddings = [doc["embedding"] for doc in embedded_documents]
reduced_embeddings = reduce_embed_size(embeddings)

for doc, reduced in zip(embedded_documents, reduced_embeddings):
  doc["embedding"] = reduced.tolist()
removeFile(TWEET_REDUCED_OUT_FILE)
with open(TWEET_REDUCED_OUT_FILE, "a", encoding="utf-8") as file:
  for doc in embedded_documents:
    file.write(json.dumps(doc, ensure_ascii=False) + "\n")

/Users/christianharjuno/anaconda3/envs/sentiment/lib/python3.9/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


OUT/indobertweet_reduced_embeds.jsonl deleted.


In [51]:
import nltk
from nltk.corpus import stopwords
import emoji
try:
  stopwords.words('english')
except LookupError:
  nltk.download('stopwords')
stopwords_combined = set(stopwords.words("indonesian")) | set(stopwords.words("english"))

def extract_structural_features(tweet):
  words = tweet.split()
  word_lengths = [len(w) for w in words]
  
  length = len(tweet)
  num_hashtags = tweet.count("#")
  num_mentions = tweet.count("@")
  num_urls = len(re.findall(r"http\S+", tweet))
  num_emojis = len([c for c in tweet if c in emoji.EMOJI_DATA])
  num_upper = sum(1 for c in tweet if c.isupper())
  num_punct = len(re.findall(r"[^\w\s]", tweet))
  avg_word_len = np.mean(word_lengths) if words else 0

  # Content/structure-oriented features
  is_question = int(tweet.strip().endswith('?'))
  is_exclamatory = int(tweet.strip().endswith('!'))
  contains_ellipsis = int("..." in tweet)
  contains_repeated_chars = int(bool(re.search(r"(.)\1{2,}", tweet)))  # e.g., sooo, yessss
  contains_short_link = int(bool(re.search(r"\b(?:https?:\/\/)?(?:www\.)?(bit\.ly|t\.co|tinyurl\.com|goo\.gl|ow\.ly|is\.gd|buff\.ly|adf\.ly|bitly\.com|cutt\.ly|rb\.gy|rebrand\.ly)\/[A-Za-z0-9]+", tweet)))
  contains_digit = int(bool(re.search(r"\d", tweet)))
  is_all_caps = int(tweet.isupper() and len(tweet) > 3)
  is_emoji_only = int(all(c in emoji.EMOJI_DATA or c.isspace() for c in tweet.strip()) and tweet.strip() != "")
  contains_quote_or_rt = int(bool(re.search(r"(RT\s@|\".+\")", tweet)))
  word_count = len(words)
  stopword_ratio = np.mean([w.lower() in stopwords_combined for w in words]) if words else 0

  return [
    length, num_hashtags, num_mentions, num_urls,
    num_emojis, num_upper, num_punct, avg_word_len,
    is_question, is_exclamatory, contains_ellipsis,
    contains_repeated_chars, contains_short_link,
    contains_digit, is_all_caps, is_emoji_only,
    contains_quote_or_rt, word_count, stopword_ratio
  ]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/christianharjuno/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Generate structural features of each cleaned content

In [74]:
import json
import numpy as np
from sklearn.preprocessing import StandardScaler
INDOBERT_NORMALIZED = OUT_DIR + "indobert_normalized.jsonl"
concat_scaler = StandardScaler()
props_scaler = StandardScaler()
with open(INDOBERT_REDUCED_OUT_FILE, "r", encoding="utf-8") as file:
  indobert_reduced_documents = [json.loads(line) for line in file]

structural_properties = [extract_structural_features(doc["cleaned_content"]) for doc in indobert_reduced_documents]

for doc, props in zip(indobert_reduced_documents, structural_properties):
  doc["structural_property"] = props
  doc["concatenated_features"] = np.concatenate([np.array(doc["embedding"]), np.array(props)])

props_scaled = props_scaler.fit_transform(np.array(structural_properties))
concat_scaled = concat_scaler.fit_transform(
    np.array([doc["concatenated_features"] for doc in indobert_reduced_documents])
)
for doc, scaled_prop, scaled_concat in zip(indobert_reduced_documents, props_scaled, concat_scaled):
  doc["structural_property"] = scaled_prop.tolist()
  doc["concatenated_features"] = scaled_concat.tolist()
  
removeFile(INDOBERT_NORMALIZED)
with open(INDOBERT_NORMALIZED, "a", encoding="utf-8") as file:
  for doc in indobert_reduced_documents:
    file.write(json.dumps(doc, ensure_ascii=False) + "\n")




OUT/indobert_normalized.jsonl deleted.


In [ ]:
import json
import numpy as np
from sklearn.preprocessing import StandardScaler
TWEET_NORMALIZED = OUT_DIR + "indobertweet_normalized.jsonl"
concat_scaler = StandardScaler()
props_scaler = StandardScaler()
with open(TWEET_REDUCED_OUT_FILE, "r", encoding="utf-8") as file:
  indobert_reduced_documents = [json.loads(line) for line in file]

structural_properties = [extract_structural_features(doc["cleaned_content"]) for doc in indobert_reduced_documents]

for doc, props in zip(indobert_reduced_documents, structural_properties):
  doc["structural_property"] = props
  doc["concatenated_features"] = np.concatenate([np.array(doc["embedding"]), np.array(props)])

props_scaled = props_scaler.fit_transform(np.array(structural_properties))
concat_scaled = concat_scaler.fit_transform(
    np.array([doc["concatenated_features"] for doc in indobert_reduced_documents])
)
for doc, scaled_prop, scaled_concat in zip(indobert_reduced_documents, props_scaled, concat_scaled):
  doc["structural_property"] = scaled_prop.tolist()
  doc["concatenated_features"] = scaled_concat.tolist()
  
removeFile(TWEET_NORMALIZED)
with open(TWEET_NORMALIZED, "a", encoding="utf-8") as file:
  for doc in indobert_reduced_documents:
    file.write(json.dumps(doc, ensure_ascii=False) + "\n")

OUT/indobertweet_normalized.jsonl does not exist.


In [78]:
def save_clustered_docs(docs, cluster_labels, output_path):
    for doc, label in zip(docs, cluster_labels):
        if "metadata" not in doc:
            doc["metadata"] = {}
        doc["metadata"]["bucket_label"] = int(label)

    # Sort by label for better organization (optional)
    docs_sorted = sorted(docs, key=lambda x: x["metadata"]["bucket_label"])

    with open(output_path, "w", encoding="utf-8") as f:
        for doc in docs_sorted:
            f.write(json.dumps(doc, ensure_ascii=False) + "\n")


### Utilize KMeans and generate buckets on indobert
1. KMeans + IndoBERT Embeddings
2. KMeans + IndoBERT Embeddings + Structure Properties
3. KMeans + Structure Properties

In [ ]:
from sre_parse import Verbose
from sklearn.cluster import KMeans
import json
import matplotlib.pyplot as plt
import numpy as np

INDOBERT_KMEANS_EMBED = OUT_DIR + "indobert-kmeans-embed.json"
INDOBERT_KMEANS_PROPS = OUT_DIR + "indobert-kmeans-props.json"
INDOBERT_KMEANS_CONCAT = OUT_DIR + "indobert-kmeans-concat.json"

with open(INDOBERT_NORMALIZED, "r", encoding="utf-8") as file:
  normalized_indobert_documents = [json.loads(line) for line in file]

embeds = [doc["embedding"] for doc in normalized_indobert_documents]
properties = [doc["structural_property"] for doc in normalized_indobert_documents]
concats = [doc["concatenated_features"] for doc in normalized_indobert_documents]

# 1. Embeddings only
kmeans_embed = KMeans(n_clusters=8, random_state=42).fit(embeds)

# 2. Embeddings + Props
kmeans_concat = KMeans(n_clusters=8, random_state=42).fit(concats)

# 3. Props only
kmeans_props = KMeans(n_clusters=8, random_state=42).fit(properties)

def save_clustered_docs(docs, cluster_labels, output_path):
    for doc, label in zip(docs, cluster_labels):
        doc["bucket_label"] = int(label)
    for doc in normalized_indobert_documents:
        if "embedding" in doc:
            del doc["embedding"]
        if "structural_property" in doc:
            del doc["structural_property"]
        if "concatenated_features" in doc:
            del doc["concatenated_features"]
            
    # Sort by label for better organization (optional)
    docs_sorted = sorted(docs, key=lambda x: x["bucket_label"])
    
    with open(output_path, "w", encoding="utf-8") as f:
      json.dump(docs_sorted, f, ensure_ascii=False)

# Cluster label predictions
labels_embed = kmeans_embed.labels_
labels_concat = kmeans_concat.labels_
labels_props = kmeans_props.labels_
# Save output files
print(len(labels_embed), len(labels_concat), len(labels_props))
  
save_clustered_docs(normalized_indobert_documents, labels_embed, INDOBERT_KMEANS_EMBED)
save_clustered_docs(normalized_indobert_documents, labels_concat, INDOBERT_KMEANS_CONCAT)
save_clustered_docs(normalized_indobert_documents, labels_props, INDOBERT_KMEANS_PROPS)

100 100 100


### Utilize KMeans and generate buckets on indobertweet
1. KMeans + IndoBERTweet Embeddings
2. KMeans + IndoBERTweet Embeddings + Structure Properties


In [ ]:
from sklearn.cluster import KMeans
import json
import matplotlib.pyplot as plt
import numpy as np

INDOBERTWEET_KMEANS_EMBED = OUT_DIR + "indobertweet-kmeans-embed.json"
INDOBERTWEET_KMEANS_PROPS = OUT_DIR + "indobertweet-kmeans-props.json"
INDOBERTWEET_KMEANS_CONCAT = OUT_DIR + "indobertweet-kmeans-concat.json"

with open(TWEET_NORMALIZED, "r", encoding="utf-8") as file:
  normalized_indobert_documents = [json.loads(line) for line in file]

embeds = [doc["embedding"] for doc in normalized_indobert_documents]
properties = [doc["structural_property"] for doc in normalized_indobert_documents]
concats = [doc["concatenated_features"] for doc in normalized_indobert_documents]

# 1. Embeddings only
kmeans_embed = KMeans(n_clusters=8, random_state=42).fit(embeds)

# 2. Embeddings + Props
kmeans_concat = KMeans(n_clusters=8, random_state=42).fit(concats)

# 3. Props only
kmeans_props = KMeans(n_clusters=8, random_state=42).fit(properties)

def save_clustered_docs(docs, cluster_labels, output_path):
    for doc, label in zip(docs, cluster_labels):
        doc["bucket_label"] = int(label)
    for doc in normalized_indobert_documents:
        if "embedding" in doc:
            del doc["embedding"]
        if "structural_property" in doc:
            del doc["structural_property"]
        if "concatenated_features" in doc:
            del doc["concatenated_features"]
            
    # Sort by label for better organization (optional)
    docs_sorted = sorted(docs, key=lambda x: x["bucket_label"])
    
    with open(output_path, "w", encoding="utf-8") as f:
      json.dump(docs_sorted, f, ensure_ascii=False)

# Cluster label predictions
labels_embed = kmeans_embed.labels_
labels_concat = kmeans_concat.labels_
labels_props = kmeans_props.labels_
# Save output files
print(len(labels_embed), len(labels_concat), len(labels_props))
  
save_clustered_docs(normalized_indobert_documents, labels_embed, INDOBERTWEET_KMEANS_EMBED)
save_clustered_docs(normalized_indobert_documents, labels_concat, INDOBERTWEET_KMEANS_CONCAT)
save_clustered_docs(normalized_indobert_documents, labels_props, INDOBERTWEET_KMEANS_PROPS)

100 100 100
